In [ ]:
import spyglass.common as sgc
import spyglass.position as sgp
import spyglass.data_import as sgi
from spyglass.utils.nwb_helper_fn import get_nwb_copy_filename
from hex_maze_behavior import populate_all_hexmaze, populate_hex_position
from fiber_photometry import populate_all_fiber_photometry

nwb_file_name = "IM-1478_20220724.nwb"

# When we run insert_session, spyglass creates a copy of the nwbfile named file name + _
# We use nwb_copy_file_name to make it explicit which one to use.
# Everything besides sgi.insert_session should use the copy file name
nwb_copy_file_name = get_nwb_copy_filename(nwb_file_name)

## Step 1. 
# Insert session into spyglass (populates all common tables)
sgi.insert_sessions(nwb_file_name)

## Step 2.
# Once the session has been inserted, insert into basic hex maze tables
# (populates HexMazeBlock, HexMazeBlock.Trial, HexMazeConfig, HexCentroids tables)
populate_all_hexmaze(nwb_copy_file_name)

## Step 3. 
# Process position using default parameters
# We only have one epoch (epoch 0) for Berke lab, so "pos 0 valid times" is always the interval list name
interval_list_name = "pos 0 valid times"

# Our position selection key includes the nwbfile name, the interval to process, and the parameters to use
position_selection_key = {
    "nwb_file_name": nwb_copy_file_name,
    "interval_list_name": interval_list_name,
    "trodes_pos_params_name": "default", # for now we always use default parameters
}
# To associate a set of parameters with a given interval, insert them into the `TrodesPosSelection` table
# Note "trodes position" really just means any imported unprocessed position
sgp.v1.TrodesPosSelection.insert1(position_selection_key, skip_duplicates=True)

# We can run the pipeline for our chosen interval/parameters by using the `TrodesPosV1.populate`
# Each NWB file, interval, and parameter set is now associated with a new analysis file and object ID.
sgp.v1.TrodesPosV1.populate(position_selection_key)

## Step 4. 
# Insert into HexPositionSelection and HexPosition tables
# (must have run populate_all_hexmaze (Step 2) and have an entry in PositionOutput table (Step 3))
populate_hex_position(nwb_copy_file_name)

## Step 5.
# Insert into all photometry-related tables
populate_all_fiber_photometry(nwb_copy_file_name)


In [ ]:
from hex_maze_behavior import HexMazeBlock, HexCentroids, HexMazeConfig, HexPositionSelection, HexPosition
from fiber_photometry import ExcitationSource, OpticalFiber, Photodetector, FiberPhotometrySeries

#key = {"nwb_file_name" : nwb_copy_file_name}
key = 'nwb_file_name LIKE "IM-%"'

print("--------------- Hex maze tables! --------------- ")

# Show the HexMazeBlock table
print("HexMazeBlock table:")
display(HexMazeBlock() & key)

# Show the Trial part table
print("HexMazeBlock().Trial part table:")
display(HexMazeBlock().Trial() & key)

# Show the HexCentroids table (nwb_file_name is key)
print("HexCentroids table:")
display(HexCentroids() & key)

# Show the HexCentroids part table (nwb_file_name and hex)
print("HexCentroidsPart")
display((HexCentroids().HexCentroidsPart()) & key)

# All config_ids for blocks in your session will also now exist in the HexMazeConfig table
# Show the HexMazeConfig table
print("HexMazeConfig table (not restricted by nwb_file_name, includes all sessions):")
display(HexMazeConfig())

# Show the HexPositionSelection table
print("HexPositionSelection table:")
display(HexPositionSelection() & key)

# Show the HexPosition table
print("HexPosition table:")
display((HexPosition()) & key)

print("--------------- Photometry tables! --------------- ")

print("OpticalFiber table (not restricted by nwb_file_name, includes all sessions):")
display(OpticalFiber() & key)

print("Photodetector table (not restricted by nwb_file_name, includes all sessions):")
display(Photodetector() & key)

print("ExcitationSource table (not restricted by nwb_file_name, includes all sessions):")
display(ExcitationSource() & key)

print("FiberPhotometrySeries table")
display(FiberPhotometrySeries() & key)

HexMazeBlock table:


nwb_file_name name of the NWB file,epoch the session epoch for this task and apparatus(1 based),block the block number within the epoch,config_id maze configuration as a string,interval_list_name descriptive name of this interval list,p_a probability of reward at port A,p_b probability of reward at port B,p_c probability of reward at port C,num_trials number of trials in this block,task_type 'barrier shift' or 'probabilty shift'
IM-1478_20220719_.nwb,0,1,"8,9,16,18,23,27,35,42,44",epoch0_block1,90.0,10.0,50.0,65,barrier change
IM-1478_20220719_.nwb,0,2,"8,9,18,21,23,27,35,42,44",epoch0_block2,90.0,10.0,50.0,62,barrier change
IM-1478_20220719_.nwb,0,3,"8,9,18,21,23,24,27,42,44",epoch0_block3,90.0,10.0,50.0,58,barrier change
IM-1478_20220720_.nwb,0,1,"10,14,15,25,26,30,34,39,46",epoch0_block1,10.0,50.0,90.0,69,probability change
IM-1478_20220720_.nwb,0,2,"10,14,15,25,26,30,34,39,46",epoch0_block2,90.0,10.0,50.0,51,probability change
IM-1478_20220724_.nwb,0,1,"11,14,15,16,28,30,32,34,45",epoch0_block1,50.0,90.0,10.0,64,probability change
IM-1478_20220724_.nwb,0,2,"11,14,15,16,28,30,32,34,45",epoch0_block2,90.0,10.0,50.0,46,probability change


HexMazeBlock().Trial part table:


nwb_file_name name of the NWB file,epoch the session epoch for this task and apparatus(1 based),block the block number within the epoch,block_trial_num trial number within the block,interval_list_name descriptive name of this interval list,epoch_trial_num trial number within the epoch,reward if the rat got a reward,"start_port A, B, or C","end_port A, B, or C","opto_cond description of opto condition, if any (delay / no_delay)","poke_interval np.array of [poke_in, poke_out]",duration trial duration in seconds
IM-1478_20220719_.nwb,0,1,1,epoch0_block1_trial1,1,0,None,C,None,=BLOB=,10.5167
IM-1478_20220719_.nwb,0,1,2,epoch0_block1_trial2,2,1,C,B,None,=BLOB=,31.1152
IM-1478_20220719_.nwb,0,1,3,epoch0_block1_trial3,3,1,B,C,None,=BLOB=,21.6368
IM-1478_20220719_.nwb,0,1,4,epoch0_block1_trial4,4,1,C,A,None,=BLOB=,54.662
IM-1478_20220719_.nwb,0,1,5,epoch0_block1_trial5,5,0,A,B,None,=BLOB=,9.01763
IM-1478_20220719_.nwb,0,1,6,epoch0_block1_trial6,6,0,B,C,None,=BLOB=,17.0026
IM-1478_20220719_.nwb,0,1,7,epoch0_block1_trial7,7,1,C,A,None,=BLOB=,81.1043
IM-1478_20220719_.nwb,0,1,8,epoch0_block1_trial8,8,0,A,B,None,=BLOB=,22.7089
IM-1478_20220719_.nwb,0,1,9,epoch0_block1_trial9,9,1,B,C,None,=BLOB=,32.461
IM-1478_20220719_.nwb,0,1,10,epoch0_block1_trial10,10,1,C,A,None,=BLOB=,18.5386


HexCentroids table:


...

config_id maze configuration as a string,len_ab number of hexes on optimal path between ports A and B,len_bc number of hexes on optimal path between ports B and C,len_ac number of hexes on optimal path between ports A and C,"path_length_diff max path length difference between lenAB, lenBC, lenAC",num_choice_points number of critical choice points for this maze config,num_cycles number of graph cycles (closed loops) for this maze config,choice_points list of hexes that are choice points (not query-able),num_dead_ends number of dead ends at least 3 hexes long,optimal_pct percentage of maze hexes that are on optimal paths,non_optimal_pct percentage of maze hexes that are on non-optimal paths,dead_end_pct percentage of maze hexes that are on dead-end paths
"10,12,18,21,25,28,34,37,42,45",23,17,19,6,1,0,=BLOB=,1,74.36,0.0,25.64
"10,14,15,25,26,30,34,39,46",21,19,15,6,1,1,=BLOB=,3,72.5,0.0,27.5
"10,14,18,19,20,23,30,42,43,46",17,17,23,6,3,2,=BLOB=,1,82.05,0.0,17.95
"10,14,18,20,23,26,29,42,43,46",19,17,21,4,3,1,=BLOB=,2,76.92,0.0,23.08
"10,14,18,20,23,26,30,42,43,46",21,17,15,6,3,1,=BLOB=,2,71.79,0.0,28.21
"10,16,18,19,20,23,30,42,43,46",15,17,21,6,3,2,=BLOB=,1,76.92,0.0,23.08
"10,16,18,19,23,30,31,42,43,46",15,17,17,2,1,1,=BLOB=,2,61.54,10.26,28.21
"11,12,14,15,18,20,22,29,31,45",17,15,17,2,1,3,=BLOB=,1,66.67,15.38,17.95
"11,12,14,15,18,22,29,31,35,45",17,17,15,2,3,4,=BLOB=,0,71.79,15.38,12.82
"11,12,14,15,20,29,31,36,45",15,17,17,2,1,3,=BLOB=,1,65.0,15.0,20.0


HexPositionSelection table:


pos_merge_id,nwb_file_name name of the NWB file,epoch the session epoch for this task and apparatus(1 based)
387dbaa5-4b21-ecfe-f585-356d112c5a76,IM-1478_20220724_.nwb,0
87fb799a-160a-115a-0624-c7125bc5a936,IM-1478_20220720_.nwb,0
b85b6071-790f-9f36-b82d-8f6ef703fb0e,IM-1478_20220719_.nwb,0


HexPosition table:


pos_merge_id,nwb_file_name name of the NWB file,epoch the session epoch for this task and apparatus(1 based),analysis_file_name name of the file,hex_assignment_object_id
387dbaa5-4b21-ecfe-f585-356d112c5a76,IM-1478_20220724_.nwb,0,IM-1478_20220724_XIV3ERCYVU.nwb,8086f214-8551-4d95-afb9-3c317d04ef85
87fb799a-160a-115a-0624-c7125bc5a936,IM-1478_20220720_.nwb,0,IM-1478_20220720_AV0UK91BYO.nwb,f01a8441-fa51-4e5d-9f42-dbe47bc943af
b85b6071-790f-9f36-b82d-8f6ef703fb0e,IM-1478_20220719_.nwb,0,IM-1478_20220719_54WSAV1ZG5.nwb,c107f021-c944-47f5-bfbd-00fb523482fd


My notes on the process for inserting into PositionOutput

In [ ]:
import spyglass.common as sgc
import spyglass.position as sgp
from spyglass.utils.nwb_helper_fn import get_nwb_copy_filename

nwb_file_name = "IM-1478_20220719.nwb"

# When we run insert_session, spyglass creates a copy of the nwbfile named file name + _
# We use nwb_copy_file_name to make it explicit which one to use
nwb_copy_file_name = get_nwb_copy_filename(nwb_file_name)

# Look at the session in the raw position table
display(sgc.common_behav.RawPosition() & {"nwb_file_name": nwb_copy_file_name})

# Look at our interval lists
display(sgc.IntervalList & {"nwb_file_name": nwb_copy_file_name})

# We only have one epoch for berke lab, so this is always the interval list name
interval_list_name = f"pos 0 valid times"
raw_position_df = (sgc.RawPosition() & {"nwb_file_name": nwb_copy_file_name, "interval_list_name": interval_list_name}).fetch1_dataframe()
display(raw_position_df)

# Default names of valid position processing params (we will use "default")
display(sgp.v1.TrodesPosParams())

# To associate a set of parameters with a given interval, insert them into the `TrodesPosSelection` table.
trodes_s_key = {
    "nwb_file_name": nwb_copy_file_name,
    "interval_list_name": interval_list_name,
    "trodes_pos_params_name": "default",
}
sgp.v1.TrodesPosSelection.insert1(
    trodes_s_key,
    skip_duplicates=True,
)

# We can run the pipeline for our chosen interval/parameters by using the `TrodesPosV1.populate`
# Each NWB file, interval, and parameter set is now associated with a new analysis file and object ID.
sgp.v1.TrodesPosV1.populate(trodes_s_key)

[2025-05-16 15:22:34,412][INFO]: DataJoint 0.14.4 connected to scrater@lmf-db.cin.ucsf.edu:3306


nwb_file_name name of the NWB file,interval_list_name descriptive name of this interval list
IM-1478_20220719_.nwb,pos 0 valid times


nwb_file_name name of the NWB file,interval_list_name descriptive name of this interval list,valid_times numpy array with start/end times for each interval,"pipeline type of interval list (e.g. 'position', 'spikesorting_recording_v1')"
IM-1478_20220719_.nwb,00_r1,=BLOB=,
IM-1478_20220719_.nwb,pos 0 valid times,=BLOB=,position
IM-1478_20220719_.nwb,raw data valid times,=BLOB=,


,xloc1,yloc1,xloc2,yloc2
time,,,,
0.712208,81.785416,88.307152,81.729721,72.904549
0.733519,82.316483,88.636253,80.011627,72.261238
0.782489,83.965561,89.037750,79.363159,73.092102
0.847202,83.222534,89.989288,80.556877,72.324982
0.910277,82.444191,94.729355,80.353867,76.612892
...,...,...,...,...
7141.946028,218.717239,142.295547,203.432556,129.844101
7142.010011,218.696426,142.388458,203.310440,129.648468
7142.073610,218.689545,141.798706,203.169769,129.280136


trodes_pos_params_name name for this set of parameters,params
decoding_xs,=BLOB=
default,=BLOB=
default_decoding,=BLOB=
default_decoding_4ms,=BLOB=
default_led0,=BLOB=
default_lowerspeed,=BLOB=
default_speed35,=BLOB=
dlc_head_body,=BLOB=
double_led,=BLOB=
double_led_CD,=BLOB=


[15:23:00][INFO] Spyglass: Computing position for: {'nwb_file_name': 'IM-1478_20220719_.nwb', 'interval_list_name': 'pos 0 valid times', 'trodes_pos_params_name': 'default'}
[15:23:22][INFO] Spyglass: Writing new NWB file IM-1478_20220719_R2UO2ONXEB.nwb
[15:23:24][INFO] Spyglass: No video frame index found. Assuming all camera frames are present.


{'success_count': 1, 'error_list': []}